In [ ]:
from math import sin, sqrt, log, exp

import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm

from preg import Preg, Logger

plt.rcParams['figure.figsize'] = [20/2.54, 16/2.54]

# 1. Linear system identification

## a. Linear system

lin_sys() defines the right hand side of a 2D linear system of the form

$$\dot{y} = A y + u $$

with a stable spiral fixed point attractor. We start with zero forcing u = 0 and let the resulting autonomous system converge to the fixed point.

In [ ]:
# zero forcing for simulating autonomous system
def zero_forcing(t):
    return 0.0

# Linear system with a stable spiral fixed point attractor.
def lin_sys(y,t, inp):
    u = inp(t)
    return np.dot(np.array([[-1, -2], [3, -1]]), np.array([y[0],y[1]])+\
        np.array([u,0]))

In [ ]:
# 1. Simulation: converge to the fixed point attractor of the autonomous
# system
n = 100                     # number of time steps
t0, t1 = 0, 10              # start and end
t = np.linspace(t0, t1, n)  # the points of evaluation of solution
h = t[1]-t[0]               # time step
y0 = [4, 1]                 # initial value
y = integrate.odeint(lin_sys, y0, t, (zero_forcing,))

plt.plot(t, y)
plt.title('convergence to fixed point attractor of autonomous system')
plt.xlabel('time [s]')
plt.grid(True)
plt.legend(('y_1', 'y_2'),loc='upper right')
plt.show()

# The attractor (we know it's simply the origin (0,0))
y_a = y[-1,:]

## b. System identification from step response

For this, we simulate the system with step forcing. The impulse response is obtained by differentiating the step response. The impulse response is identical to the first-order Volterra kernel of the system.

In [ ]:
def step_forcing(t):
    if t >= 0:
        return 1.0
    return 0.0

In [ ]:
# 2. Force the system by a step-wise input u(t) being initially situated on
# the attractor
y_step = integrate.odeint(lin_sys, y_a, t, (step_forcing,))

plt.plot(t, y_step)
plt.title('step response')
plt.xlabel('time [s]')
plt.grid(True)
plt.legend(('y_1', 'y_2'),loc='upper right')
plt.show()

In [ ]:
# The linear or first-order (1) impulse response function (IRF), a.k.a.
# Volterra kernel: http://www.scholarpedia.org/article/Volterra_and_Wiener_series
h_1 = np.diff(y_step[:,0], prepend=0)

plt.plot(t, h_1)
plt.title('impulse response $y_1$')
plt.xlabel('time [s]')
plt.grid(True)
plt.show()

## c. Test the impulse response on a sine input

In [ ]:
def sine_forcing(t):
    om = 1
    return sin(om*t)

In [ ]:
# 3. Test the prediction performed by using the IRF; apply any desirable
# forcing
y_sine = integrate.odeint(lin_sys, y_a, t, (sine_forcing,))
u_sine = np.array([sine_forcing(t2) for t2 in t])
y_pr1 = np.convolve(u_sine, h_1, 'full')
y_pr1 = y_pr1[:n]

plt.plot(t, y_sine[:,0], '-o', t, y_pr1, '-x')
plt.title('sine response')
plt.xlabel('time [s]')
plt.grid(True)
plt.legend(('true model output', 'IRF prediction'),loc='upper right')
plt.show()

## d. System identification via linear regression

Linear regression needs data in the form of input-output pairs $(x_i,y_i)$. $y_i = y(t_i)$ is the system output at time $t_i$, $x_i = (x(t_{i-m}), x(t_{i-m+1}), \dots, x(t_{i-1}), x(t_i))$ the current input and all past inputs that affect the system output. $m$ is the memory of the system. This data format is obtained by sliding a window of size $m$ over the input time series, the associated output is the element of the output time series that corresponds to the last input of the window.

In [ ]:
# convert step input to sliding window format for regression (see 4.)
memory = n # how may sampling points back in time are used as input for the Volterra operators
u1 = np.zeros(n + memory - 1)
u1[memory:] = np.ones(n-1) # create step input by prepending zeros
x0 = np.zeros((n, memory))
for i in range(0,n):
    x0[i,:] = u1[i:i + memory]
y0 = y_step[:,0] # associated output

# convert sine input to sliding window format for regression (see 4.)
u2 = np.zeros(n + memory - 1)
u2[memory-1:] = u_sine
x1 = np.zeros((n, memory))
for i in range(0,n):
    x1[i,:] = u2[i:i + memory]

We train and test on the same time series as above, converted to the sliding window format. The first-order Volterra operator obtained from *preg* is given as coefficient vector $\eta = (\eta_0, \eta_1, \dots, \eta_m)$ such that the system output is computed as
$$ y_i = \sum_{j=0}^m \eta_{m-j} x_{i-j}, $$
so the coefficients from *preg* just keep the same order as the input window. To obtain the classical convolution notation 
$$ y_i = \sum_{j=i-m}^i x_j \eta'_{m-j} , $$
the first-order kernel must be flipped, i.e., $\eta'_j = \eta_{m-j}$. The flipped kernel is identical to the impulse response.

In [ ]:
# 4. Obtain h_1 using poly_reg
with Logger('linear') as lg: # get a logger instance

    ptype = 'ihp'		# kernel type ('ihp' or 'ap')
    method = 'gpp'		# model selection method ('llh', 'gpp', 'loo')
    n_iter = 20		    # number of iterations
    order = 1           # Volterra series order

    # regression
    hp0 = [log(0.6), log(sqrt(0.001))]
    gp = Preg(lg, ptype, order, hp0) # init GP struct
    gp.ams(x0, y0, method, n_iter)     # do regression
    mu_step = gp.predict(x0)    # predict on training input

    # plot prediction on training set (step function)
    plt.plot(t, mu_step, '-o', t, y0, '-x')
    plt.title('Linear prediction on training set')
    plt.xlabel('time [s]')
    plt.legend(('preg prediction','true model output'),loc='upper right')
    plt.show()

    # prediction on test set (sine)
    mu_sine = gp.predict(x1)
    plt.plot(t, mu_sine, '-o', t, y_sine[:,0], '-x')
    plt.title('Linear prediction on test set')
    plt.xlabel('time [s]')
    plt.legend(('preg prediction','true model output'),loc='upper right')
    plt.show()

    # compute explicit 1st-order Volterra operator
    eta = gp.volt(order)
    eta = np.flip(eta) # flip to obtain impulse response
    plt.plot(t[:n-1], eta[:n-1], '-o', t[:n-1], np.diff(y_step[:,0]), '-x')
    plt.title('First order Volterra kernel')
    plt.xlabel('Delay [s]')
    plt.legend(('preg','impulse response'),loc='upper right')
    plt.show()

# Nonlinear system identification

## Second-order nonlinear system

Oor second-order nonlinear system example is simply the linear system from above where the output is fed through a $x^2()$ nonlinearity:
$$\dot{y} = (A y + u)^2. $$
Since we want to test our regression on a smooth sine input, it is useful to obtain our training data from a time series with a similar degree of smoothness. Here, we just concatenate flipped and unflipped versions of the previous step response as our training input time series and feed it through the nonlinear system.

In [ ]:
# simulate 2nd-order nonlinear system with smooth input
xtr = np.concatenate((y_step[:,1], np.flip(y_step[:,1]), y_step[:,1],))
ntr = len(xtr)
y_nl = np.convolve(xtr, h_1, 'full')
y_nl = y_nl[:ntr+1]
y_nl = y_nl**2
ttr = np.arange(0,h*(ntr - memory + 1),h)
x2 = np.zeros((len(ttr), memory))
y2 = np.zeros((len(ttr),))
for i in range(0,len(y2)):
    x2[i,:] = xtr[i:i + memory]
    y2[i] = y_nl[i + memory - 1]

Again, *preg* gives the second-order Volterra kernel in the form
$$ y_i = \sum_{j=0}^m \sum_{k=0}^m \eta_{m-j,m-k} x_{i-j}x_{i-k} $$. 
For the tradiational notation, the resulting 2D kernel needs to be flipped in both temporal directions $\eta'_{jk} = \eta_{m-j,m-k}$ such that
$$ y_i = \sum_{j=i-m}^i \sum_{k=i-m}^i \eta_{m-j,m-k} x_j x_k $$. 

In [ ]:
# 5. Obtain second-order Volterra kernel for a nonlinear operator
# do regression and predict on training data
with Logger('nonlinear') as lg: # get a logger instance

    order = 2           # Volterra series order

    # regression
    hp0 = [log(0.6), log(sqrt(0.001))]
    gp_nl = Preg(lg, ptype, order, hp0) # init GP struct
    gp_nl.ams(x2, y2, method, n_iter)     # do regression
    mu_nl = gp_nl.predict(x2)    # predict on training input

    # plot prediction on training set
    plt.plot(ttr, mu_nl, ttr, y2) #, ttr, xtr[memory-1:memory+len(ttr)-1])
    plt.title('Nonlinear prediction on training set')
    plt.xlabel('time [s]')
    plt.legend(('prediction','true model output','input'),loc='upper right')
    plt.show()

    # prediction on test set (sine)
    mu_sine_nl = gp_nl.predict(x1)
    plt.plot(t, mu_sine_nl, '-o', t, y_sine[:,0]**2, '-x')
    plt.title('Nonlinear prediction on test set')
    plt.xlabel('time [s]')
    plt.legend(('prediction','true model output'),loc='upper right')
    plt.show()

    # 2nd-order Volterra kernel
    eta = gp_nl.volt(order)
    eta = np.flipud(np.fliplr(eta)) # time delay is measured in reverse time direction
                                    # kernel mst be flipped for standrad display
                                    # convention
    xv, yv = np.meshgrid(t, t)
    plt.rcParams['figure.figsize'] = [30/2.54, 24/2.54]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(xv, yv, eta, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    ax.set_xlabel('delay [s]')
    ax.set_ylabel('delay [s]')
    plt.title('Second order Volterra kernel')
    plt.show()